In [12]:
import pandas as pd
import numpy as np
import math

In [13]:
data = pd.read_csv('naivedata.csv')
data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [14]:
train_set = data.head(int(len(data)*0.7))
train_set

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
531,1,86,66,52,65,41.3,0.917,29,0
532,6,91,0,0,0,29.8,0.501,31,0
533,1,77,56,30,56,33.3,1.251,24,0
534,4,132,0,0,0,32.9,0.302,23,1


In [15]:
test_set = data.loc[data.index > len(train_set)-1]
test_set

,6,148,72,35,0,33.6,0.627,50,1
536,0,57,60,0,0,21.7,0.735,67,0
537,0,127,80,37,210,36.3,0.804,23,0
538,3,129,92,49,155,36.4,0.968,32,1
539,8,100,74,40,215,39.4,0.661,43,1
540,3,128,72,25,190,32.4,0.549,27,1
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [16]:
p1 = len(data.where(data[data.columns.tolist()[-1]] == 1).dropna())
n1 = len(data.where(data[data.columns.tolist()[-1]] == 0).dropna())
p=p1/(p1+n1)
n=n1/(p1+n1)
print(p,n)

0.34810951760104303 0.651890482398957


In [17]:
attributes = data.columns.tolist()
attributes = attributes[0:-1]
attributes

['6', '148', '72', '35', '0', '33.6', '0.627', '50']

In [18]:
values = {}
for i in attributes:
    values[i] = data[i].unique()

In [19]:
from collections import defaultdict
#function to create a multidimensional dictionary
def multi_dict(K, type):
    if K == 1:
        return defaultdict(type)
    else:
        return defaultdict(lambda: multi_dict(K-1, type))

In [20]:
curr_prob = {}
curr_prob = multi_dict(3, float)
for i in attributes:
    sum=0
    for j in values[i]:
        curr_prob_yes=len(data.where((data[i]==j) & (data[data.columns.tolist()[-1]]==1)).dropna())
        curr_prob_no=len(data.where((data[i]==j) & (data[data.columns.tolist()[-1]]==0)).dropna())
        curr_prob[i][j]['yes']=curr_prob_yes/p1
        curr_prob[i][j]['no']=curr_prob_no/n1

In [21]:
def test_model(test_data):
    unsuccessfull_count = 0
    for index,i in test_data.iterrows():
        v_yes,v_no,res = p,n,-1
        x=0
        for j in attributes:
            v_yes *= curr_prob[j][i[x]]['yes']
            v_no *= curr_prob[j][i[x]]['no']
            x+=1
        v_yes = v_yes/(v_yes + v_no)
        v_no = v_no/(v_yes + v_no)
        if(v_yes >= v_no):
            res=1
        else:
            res=0
        if(res != i[-1]):
            unsuccessfull_count += 1
    print("Accuracy of Naive Bayes classifier : ",((len(test_data)-unsuccessfull_count)/len(test_data))*100,"%")
test_model(test_set)

Accuracy of Naive Bayes classifier :  93.07359307359307 %
